<a href="https://colab.research.google.com/github/georgesteve/Photomosaic-sgac/blob/main/SGAC_Photographic_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob
from PIL import Image
from scipy import spatial
import numpy as np

#Photographic Mosaic

In [ ]:
Image.MAX_IMAGE_PIXELS = None

In [ ]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [ ]:
ls

drive/  sample_data/


In [ ]:
cd drive/MyDrive

/content/drive/MyDrive


In [ ]:
# Sources and settings
#main_photo_path = "IGP-y-volcán-Misti.png"
#main_photo_path = "noche-estrellada-index.jpeg"
#main_photo_path = "Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpeg"
#main_photo_path = "86376680_130511965138315_457165819453898752_n.jpg"
#main_photo_path = "69563.jpg"
#main_photo_path = "1_MKE4QgF0BdA6fPM_mx6bpg.jpeg"
main_photo_path = "2728px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg"


#tile_photos_path = "Ale-pixel/thumbnails/*"

#tile_photos_path = "Fotos Ale/WhatsApp Unknown 2022-07-17 at 2.54.19 PM/*"
tile_photos_path = 'El espacio es de todos, todos somos el espacio.  (File responses)/Comparte tu foto aquí por favor. Share your photo here please. (File responses)/*'
tile_size = (15,15)

In [ ]:
# Get all tiles
tile_paths = []
for file in glob.glob(tile_photos_path):
    tile_paths.append(file)

In [ ]:
print(tile_paths)

['El espacio es de todos, todos somos el espacio.  (File responses)/Comparte tu foto aquí por favor. Share your photo here please. (File responses)/WhatsApp Image 2022-08-06 at 9.47.21 PM.jpeg', 'El espacio es de todos, todos somos el espacio.  (File responses)/Comparte tu foto aquí por favor. Share your photo here please. (File responses)/WhatsApp Image 2022-08-06 at 9.47.08 PM (1).jpeg', 'El espacio es de todos, todos somos el espacio.  (File responses)/Comparte tu foto aquí por favor. Share your photo here please. (File responses)/WhatsApp Image 2022-08-06 at 9.47.08 PM.jpeg', 'El espacio es de todos, todos somos el espacio.  (File responses)/Comparte tu foto aquí por favor. Share your photo here please. (File responses)/WhatsApp Image 2022-08-06 at 9.46.42 PM.jpeg', 'El espacio es de todos, todos somos el espacio.  (File responses)/Comparte tu foto aquí por favor. Share your photo here please. (File responses)/WhatsApp Image 2022-08-06 at 9.47.42 PM.jpeg', 'El espacio es de todos, 

In [ ]:
# Import and resize all tiles
tiles = []
for path in tile_paths:
    tile = Image.open(path)
    tile = tile.resize(tile_size)
    tiles.append(tile)

In [ ]:
# Calculate dominant color
colors = []
for tile in tiles:
    mean_color = np.array(tile).mean(axis=0).mean(axis=0)
    
    if np.shape(mean_color)==(3,):
        colors.append(mean_color)
#print(colors)
        #print(np.shape(mean_color))

In [ ]:
# Pixelate (resize) main photo
main_photo = Image.open(main_photo_path)
width = int(np.round(main_photo.size[0] / tile_size[0]))
height = int(np.round(main_photo.size[1] / tile_size[1]))

resized_photo = main_photo.resize((width, height))

In [ ]:
#colors = np.asarray(colors,dtype='object')

# Find closest tile photo for every pixel

# Create a KDTree
tree = spatial.KDTree(colors)

# Empty integer array to store indices of tiles
closest_tiles = np.zeros((width, height), dtype=np.uint32)

for i in range(width):
  for j in range(height):
    pixel=resized_photo.getpixel((i, j))
    
    closest=tree.query(pixel)   
    #print(pixel,colors[closest[1]])
    closest_tiles[i, j] = closest[1]

In [ ]:
# Create an output image
output = Image.new('RGB', (main_photo.size[0],main_photo.size[1]))

# Draw tiles
for i in range(width):
    for j in range(height):
        # Offset of tile
        x, y = i*tile_size[0], j*tile_size[1]
        # Index of tile
        index = closest_tiles[i, j]
        # Draw tile
        output.paste(tiles[index], (x, y))

# Save output
output.save("SGC-art-output.pdf")